In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.6)
name = llm.invoke("I want to open a restaurant for Italian food. Suggest a fancy name for this.")
print(name)



"La Dolce Vita Trattoria"


In [3]:
from langchain.prompts import PromptTemplate

prompt_restaurant_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

prompt_restaurant_name.format(cuisine='French')

'I want to open a restaurant for French food. Suggest a fancy name for this.'

In [4]:
from langchain.chains import LLMChain

name_chain = LLMChain(llm=llm, prompt=prompt_restaurant_name)

In [5]:
prompt_menu_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}. Return it as a comma separated list."
)
menu_items_chain = LLMChain(llm=llm, prompt=prompt_menu_items)

In [6]:
from langchain.chains import SimpleSequentialChain

sim_seq_chain = SimpleSequentialChain(chains=[name_chain, menu_items_chain])
response = sim_seq_chain.invoke("French")
print(response)

{'input': 'French', 'output': "\n\nLa Belle Cuisine:\n1. Coq au vin\n2. Beef bourguignon\n3. Ratatouille\n4. Escargots\n5. Bouillabaisse\n6. Croque monsieur\n7. Quiche Lorraine\n8. Steak frites\n9. Salade niçoise\n10. Crème brûlée\n\nLe Petit Bistro:\n1. Boeuf bourguignon sandwich\n2. Croque madame\n3. Salade lyonnaise\n4. Moules frites\n5. Quiche du jour\n6. Cassoulet\n7. Poulet rôti\n8. Tarte tatin\n9. Soupe à l'oignon gratinée\n10. Profiteroles au chocolat"}


In [7]:
from langchain.chains import SequentialChain

name_chain = LLMChain(llm=llm, prompt=prompt_restaurant_name, output_key="restaurant_name")
menu_items_chain = LLMChain(llm=llm, prompt=prompt_menu_items, output_key="menu_items")
seq_chain = SequentialChain(
    chains=[name_chain, menu_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaurant_name', 'menu_items']
)

In [8]:
seq_chain.invoke({'cuisine': 'Italian'})

{'cuisine': 'Italian',
 'restaurant_name': '\n\n"Presto Italiano"',
 'menu_items': '\n\n1. Margherita pizza\n2. Fettuccine Alfredo\n3. Chicken Parmigiana\n4. Lasagna\n5. Spaghetti Bolognese\n6. Caprese salad\n7. Cannelloni\n8. Garlic bread\n9. Tiramisu\n10. Gelato\n11. Bruschetta\n12. Ravioli\n13. Penne Arrabiata\n14. Calamari Fritti\n15. Eggplant Parmigiana'}